# Análise de tópicos

In [16]:
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
import pandas as pd
import pyarrow
import string
from unidecode import unidecode
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from nltk.stem.porter import PorterStemmer
import numpy as np

In [2]:
import nltk
nltk.download('rslp')

[nltk_data] Downloading package rslp to /home/luiz-guilherme-
[nltk_data]     picinin/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [2]:
# leitura de arquivos
articles = pd.read_parquet(path='../data/articles2.parquet', engine='pyarrow')

In [3]:
articles

,title,title_alt,authors,advisor1,keywords,date,publisher,campus,citation,resumo,abstract,uri,pdf_uri
unique_id,,,,,,,,,,,,,
28320,Validação de modelos utilizando problemas inve...,None,"[Silva, Vinícius Vitorelli Generoso da]","Koroishi, Edson Hideki","[Problemas inversos (Equações diferenciais), V...",2015-02-11,Universidade Tecnológica Federal do Paraná,Cornelio Procopio,"SILVA, Vinícius Vitorelli Generoso da. Validaç...",Existe uma dificuldade desde os primórdios da ...,From the beginning there is difficulty in engi...,http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
36735,Destinação do lodo de estação de tratamento de...,Disposal of water treatment plant sludge in wa...,"[Tamankievies, Daniela Fatima]","Bortoli, Marcelo","[Água - Estações de tratamento, Lodo, Testes d...",2025-03-27,Universidade Tecnológica Federal do Paraná,Francisco Beltrao,"TAMANKIEVIES, Daniela Fatima. Destinação do lo...",O lodo gerado em Estações de Tratamento de Águ...,The sludge generated in Water Treatment Plants...,http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
36437,Gramep: método livre de alinhamento baseado no...,Gramep: an alignment-free method based on the ...,"[Zanon, Matheus Henrique Pimenta]","Lopes, Fabricio Martins","[Genômica, Aprendizado do computador, Classifi...",2025-03-27,Universidade Tecnológica Federal do Paraná,Cornelio Procopio,"ZANON, Matheus Henrique Pimenta. Gramep: métod...",Com o avanço na tecnologia de sequenciamento d...,"With advances in DNA sequencing technology, th...",http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
36652,Frações na perspectiva de medição e a abordage...,Fractions from the measurement perspective and...,"[Shiinoki, Vanessa Garcia]","Rocha, Zenaide de Fatima Dante Correia","[Matemática - Estudo e ensino, Frações, Mediçã...",2025-03-27,Universidade Tecnológica Federal do Paraná,Londrina,"SHIINOKI, Vanessa Garcia. Frações na perspecti...",O objetivo desta pesquisa foi analisar os proc...,The objective of this research was to analyze ...,http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
37105,Consciência fonológica na alfabetização: propo...,Phonological awareness in the literacy: a prop...,"[Rosa, Daniela Dala]","Denardi, Didie Ana Ceni","[Língua inglesa, Fonologia, Literacia, Ensino ...",2025-03-28,Universidade Tecnológica Federal do Paraná,Pato Branco,"ROSA, Daniela Dala. Consciência fonológica na ...","Esta pesquisa explicatória (Gil, 2021) versa s...","This exploratory research (Gil, 2021) deals wi...",http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26646,Vibração livre e flambagem de placas de Mindli...,Free-vibration and buckling analysis of Mindli...,"[Araujo, Leilson Joaquim]","Abdalla Filho, João Elias","[Vibração, Método dos elementos finitos, Flamb...",2021-05-28,Universidade Tecnológica Federal do Paraná,Curitiba,"ARAUJO, Leilson Joaquim. Vibração livre e flam...",O método dos elementos finitos (MEF) é um méto...,The finite element method (FEM) is an approxim...,http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
25542,Equilíbrio de fases experimental de hidratos d...,Experimental phase equilibria of carbon dioxid...,"[Vasconcelos, Luiz Fernando Santos de]","Marcelino Neto, Moises Alves","[Hidratos, Dióxido de carbono, Metano, Cloreto...",2021-05-28,Universidade Tecnológica Federal do Paraná,Curitiba,"VASCONCELOS, Luiz Fernando Santos de. Equilíbr...","Em certas condições termodinâmicas, as molécul...","Under certain thermodynamic conditions, water ...",http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
26006,Influência das etapas de branqueamento e tempe...,Influence of bleaching steps 

### Pré-processamento

In [4]:
# resumos = articles['resumo']
resumos = articles['abstract']
resumos

unique_id
28320    From the beginning there is difficulty in engi...
36735    The sludge generated in Water Treatment Plants...
36437    With advances in DNA sequencing technology, th...
36652    The objective of this research was to analyze ...
37105    This exploratory research (Gil, 2021) deals wi...
                               ...                        
26646    The finite element method (FEM) is an approxim...
25542    Under certain thermodynamic conditions, water ...
26006    Yerba mate (Ilex paraguariensis) is a plant th...
28828    Two-phase flows are found in several industria...
25530    Urban agglomerations are a growing phenomenon ...
Name: abstract, Length: 17780, dtype: object

In [5]:
# conversão para minúsculo
resumos_prep = resumos.str.lower()
resumos_prep

unique_id
28320    from the beginning there is difficulty in engi...
36735    the sludge generated in water treatment plants...
36437    with advances in dna sequencing technology, th...
36652    the objective of this research was to analyze ...
37105    this exploratory research (gil, 2021) deals wi...
                               ...                        
26646    the finite element method (fem) is an approxim...
25542    under certain thermodynamic conditions, water ...
26006    yerba mate (ilex paraguariensis) is a plant th...
28828    two-phase flows are found in several industria...
25530    urban agglomerations are a growing phenomenon ...
Name: abstract, Length: 17780, dtype: object

In [6]:
# remoção de pontuação
PONTUACAO = string.punctuation

indexes = resumos_prep.index
i = 0

for resumo in resumos_prep:
    index = indexes[i]
    if resumo is not None:
        resumos_prep[index] = unidecode(resumo.translate(str.maketrans('', '', PONTUACAO)))
    else:
        resumos_prep[index] = ''
    i = i + 1
    
resumos_prep

unique_id
28320    from the beginning there is difficulty in engi...
36735    the sludge generated in water treatment plants...
36437    with advances in dna sequencing technology the...
36652    the objective of this research was to analyze ...
37105    this exploratory research gil 2021 deals with ...
                               ...                        
26646    the finite element method fem is an approximat...
25542    under certain thermodynamic conditions water m...
26006    yerba mate ilex paraguariensis is a plant that...
28828    twophase flows are found in several industrial...
25530    urban agglomerations are a growing phenomenon ...
Name: abstract, Length: 17780, dtype: object

In [7]:
# remoção de stopwords
# STOPWORDS = set(stopwords.words('portuguese'))
STOPWORDS = set(stopwords.words('english'))

def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

resumos_prep = resumos_prep.apply(lambda text: remove_stopwords(text))

print(resumos_prep)

unique_id
28320    beginning difficulty engineering obtaining phy...
36735    sludge generated water treatment plants wtps p...
36437    advances dna sequencing technology volume avai...
36652    objective research analyze mathematical reason...
37105    exploratory research gil 2021 deals phonologic...
                               ...                        
26646    finite element method fem approximate method s...
25542    certain thermodynamic conditions water molecul...
26006    yerba mate ilex paraguariensis plant leaves bl...
28828    twophase flows found several industrial proces...
25530    urban agglomerations growing phenomenon world ...
Name: abstract, Length: 17780, dtype: object


In [8]:
# stemming
# stemmer = RSLPStemmer()
stemmer = PorterStemmer()

def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

resumos_prep = resumos_prep.apply(lambda text: stem_words(text))
resumos_prep

unique_id
28320    begin difficulti engin obtain physic quantiti ...
36735    sludg gener water treatment plant wtp pose sig...
36437    advanc dna sequenc technolog volum avail genom...
36652    object research analyz mathemat reason process...
37105    exploratori research gil 2021 deal phonolog aw...
                               ...                        
26646    finit element method fem approxim method solut...
25542    certain thermodynam condit water molecul combi...
26006    yerba mate ilex paraguariensi plant leav bleac...
28828    twophas flow found sever industri process oil ...
25530    urban agglomer grow phenomenon world along pop...
Name: abstract, Length: 17780, dtype: object

### Detecção de tópicos

In [12]:
NUM_TOPICS = 10

dataset_orig = [d.split() for d in resumos_prep]
dictionary_orig = Dictionary(dataset_orig)
corpus_orig = [dictionary_orig.doc2bow(doc) for doc in dataset_orig]

model_original =LdaModel(corpus=corpus_orig, id2word=dictionary_orig, num_topics=NUM_TOPICS, iterations=500, passes=5,random_state=1)

In [13]:
for i in  model_original.show_topics(num_topics=NUM_TOPICS, num_words=6, log=False):
    print(i)
    print('---')

(0, '0.012*"plant" + 0.012*"de" + 0.011*"soil" + 0.011*"evalu" + 0.010*"use" + 0.009*"speci"')
---
(1, '0.013*"studi" + 0.012*"environment" + 0.009*"work" + 0.009*"use" + 0.009*"wast" + 0.009*"water"')
---
(2, '0.017*"treatment" + 0.016*"concentr" + 0.015*"use" + 0.013*"extract" + 0.012*"remov" + 0.012*"compound"')
---
(3, '0.021*"use" + 0.018*"model" + 0.015*"method" + 0.013*"result" + 0.010*"perform" + 0.009*"test"')
---
(4, '0.016*"product" + 0.014*"use" + 0.014*"process" + 0.013*"compani" + 0.013*"manag" + 0.012*"develop"')
---
(5, '0.015*"educ" + 0.015*"research" + 0.014*"student" + 0.012*"work" + 0.011*"school" + 0.011*"use"')
---
(6, '0.015*"group" + 0.010*"diseas" + 0.008*"age" + 0.007*"patient" + 0.005*"train" + 0.005*"sport"')
---
(7, '0.020*"product" + 0.011*"use" + 0.010*"sampl" + 0.008*"food" + 0.008*"content" + 0.008*"result"')
---
(8, '0.043*"system" + 0.022*"use" + 0.020*"control" + 0.019*"energi" + 0.016*"electr" + 0.015*"develop"')
---
(9, '0.018*"materi" + 0.017*"use

### Probabilidades

In [14]:
docsProbabilities = model_original.get_document_topics(corpus_orig, minimum_probability=0)

docsProbabilities

In [15]:
#Probability of each topic per document. Tuple (topic, probability)
for docDistribution in docsProbabilities[0:10]:
    print(docDistribution)

[(0, 0.0012052307), (1, 0.0012054126), (2, 0.0012052464), (3, 0.9891514), (4, 0.001205623), (5, 0.0012054399), (6, 0.0012051559), (7, 0.0012054474), (8, 0.0012053988), (9, 0.0012056775)]
[(0, 0.00070676906), (1, 0.1050916), (2, 0.2895149), (3, 0.0007067489), (4, 0.20211455), (5, 0.047668602), (6, 0.010223659), (7, 0.34255975), (8, 0.00070674426), (9, 0.0007067188)]
[(0, 0.033451445), (1, 0.00056802), (2, 0.040214345), (3, 0.6168968), (4, 0.29923496), (5, 0.00056797947), (6, 0.007362537), (7, 0.0005679821), (8, 0.0005680316), (9, 0.0005679164)]
[(0, 0.0006377734), (1, 0.01993904), (2, 0.0006379206), (3, 0.113108516), (4, 0.0006378306), (5, 0.7456858), (6, 0.048370518), (7, 0.0006378324), (8, 0.0006377744), (9, 0.069706924)]
[(0, 0.0004155911), (1, 0.00041570762), (2, 0.00041558573), (3, 0.00041561353), (4, 0.000415607), (5, 0.98553824), (6, 0.0004156216), (7, 0.00041558602), (8, 0.011136787), (9, 0.0004156216)]
[(0, 0.00043856318), (1, 0.50834656), (2, 0.00043856152), (3, 0.00043856548)

In [ ]:
docsProbabilities2 = []
for doc in docsProbabilities:
    aux = []
    for topic in doc:
        aux.append(topic[1])
    docsProbabilities2.append(aux)


In [22]:
columns_names = []
for i in range(0, NUM_TOPICS):
    columns_names.append(f"topic{i}")

columns_names

['topic0',
 'topic1',
 'topic2',
 'topic3',
 'topic4',
 'topic5',
 'topic6',
 'topic7',
 'topic8',
 'topic9']

In [24]:
dfProbs = pd.DataFrame(data=np.array(docsProbabilities2), columns=columns_names, index=indexes)

dfProbs

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9
unique_id,,,,,,,,,,
28320,0.001205,0.001205,0.001205,0.989151,0.001206,0.001205,0.001205,0.001205,0.001205,0.001206
36735,0.000707,0.105074,0.289517,0.000707,0.202123,0.047671,0.010224,0.342564,0.000707,0.000707
36437,0.033451,0.000568,0.040214,0.616895,0.299237,0.000568,0.007363,0.000568,0.000568,0.000568
36652,0.000638,0.019943,0.000638,0.113101,0.000638,0.745686,0.048371,0.000638,0.000638,0.069709
37105,0.000416,0.000416,0.000416,0.000416,0.000416,0.985540,0.000416,0.000416,0.011135,0.000416
...,...,...,...,...,...,...,...,...,...,...
26646,0.017020,0.035719,0.023338,0.494648,0.000580,0.044949,0.000580,0.019354,0.000580,0.363231
25542,0.070967,0.019518,0.335075,0.308509,0.040819,0.000517,0.000517,0.018629,0.000517,0.204933
26006,0.066106,0.000363,0.337984,0.109620,0.000363,0.000363,0.023440,0.417143,0.007894,0.036725


In [25]:
dfResumos = pd.DataFrame(data=resumos_prep, columns=['abstract'], index=indexes)

dfResumos

,abstract
unique_id,
28320,begin difficulti engin obtain physic quantiti ...
36735,sludg gener water treatment plant wtp pose sig...
36437,advanc dna sequenc technolog volum avail genom...
36652,object research analyz mathemat reason process...
37105,exploratori research gil 2021 deal phonolog aw...
...,...
26646,finit element method fem approxim method solut...
25542,certain thermodynam condit water molecul combi...
26006,yerba mate ilex paraguariensi plant leav bleac...


In [26]:
dfMix = dfResumos.join(dfProbs)

dfMix

,abstract,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9
unique_id,,,,,,,,,,,
28320,begin difficulti engin obtain physic quantiti ...,0.001205,0.001205,0.001205,0.989151,0.001206,0.001205,0.001205,0.001205,0.001205,0.001206
36735,sludg gener water treatment plant wtp pose sig...,0.000707,0.105074,0.289517,0.000707,0.202123,0.047671,0.010224,0.342564,0.000707,0.000707
36437,advanc dna sequenc technolog volum avail genom...,0.033451,0.000568,0.040214,0.616895,0.299237,0.000568,0.007363,0.000568,0.000568,0.000568
36652,object research analyz mathemat reason process...,0.000638,0.019943,0.000638,0.113101,0.000638,0.745686,0.048371,0.000638,0.000638,0.069709
37105,exploratori research gil 2021 deal phonolog aw...,0.000416,0.000416,0.000416,0.000416,0.000416,0.985540,0.000416,0.000416,0.011135,0.000416
...,...,...,...,...,...,...,...,...,...,...,...
26646,finit element method fem approxim method solut...,0.017020,0.035719,0.023338,0.494648,0.000580,0.044949,0.000580,0.019354,0.000580,0.363231
25542,certain thermodynam condit water molecul combi...,0.070967,0.019518,0.335075,0.308509,0.040819,0.000517,0.000517,0.018629,0.000517,0.204933
26006,yerba mate ilex paraguariensi plant leav bleac...,0.066106,0.000363,0.337984,0.109620,0.000363,0.000363,0.023440,0.417143,0.007894,0.036725


### Detecção de anomalias a partir dos tópicos

In [39]:
dfMix2 = dfMix.copy()

for index, row in dfMix2.iterrows():
    if row['abstract'] == '':
        dfMix2 = dfMix2.drop(index=index)
        continue
        
    for col in columns_names:
        if row[col] >= 0.25:
            dfMix2 = dfMix2.drop(index=index)
            break
            

print(f"de {dfMix['abstract'].count()} foi para {dfMix2['abstract'].count()}")
dfMix2

de 17780 foi para 72


,abstract,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9
unique_id,,,,,,,,,,,
36547,blood flow restrict bfr appli resist train rt ...,0.000836,0.192223,0.025713,0.203797,0.097409,0.000836,0.153522,0.107381,0.217449,0.000836
36774,sewag treatment involv sever step remov impur ...,0.000817,0.243136,0.212990,0.044037,0.164033,0.012463,0.000817,0.091311,0.000817,0.229580
36705,soil one main regul life earth essenti human s...,0.236196,0.188512,0.000562,0.245928,0.231957,0.000562,0.026618,0.000562,0.000562,0.068540
36242,gasif process whose applic began late 17th cen...,0.000835,0.000835,0.105359,0.237252,0.220572,0.171434,0.009600,0.125541,0.058293,0.070280
36193,fungal diseas repres one major threat global a...,0.024844,0.000377,0.241305,0.000377,0.000377,0.000377,0.244566,0.241824,0.000377,0.245577
...,...,...,...,...,...,...,...,...,...,...,...
11515,guarante autonomi rural produc form mixtur pes...,0.101566,0.051450,0.001235,0.236457,0.190577,0.069208,0.001235,0.195390,0.001235,0.151646
24214,accord american academi orthoped postur defin ...,0.026632,0.139455,0.000737,0.241776,0.065965,0.183325,0.161380,0.000737,0.121065,0.058928
4997,work aim develop softwar enabl use digit imag ...,0.000618,0.000618,0.000618,0.233356,0.199188,0.212337,0.000618,0.149777,0.202253,0.000618


In [42]:
articles['id'] = articles.index
dfMix2['id'] = dfMix2.index

In [47]:
for index, row in articles.iterrows():
    if row['id'] in dfMix2.index:
        print(row['title'])
        print('-----------------------------------------------------------------------------------------')

Protótipo de sistema integrado de controle da restrição de fluxo sanguíneo no exercício físico
-----------------------------------------------------------------------------------------
Produtividade de mão de obra em tanques de concreto armado: estudo de caso: ETE Pato Branco
-----------------------------------------------------------------------------------------
Avaliação de mapas anuais de estoque de carbono orgânico do solo brasileiro (1985-2023)
-----------------------------------------------------------------------------------------
Proposta de metodologia para simulação dinâmica do processo de gaseificação contracorrente utilizando softwares livres
-----------------------------------------------------------------------------------------
Síntese de vidros polifosfato e borofosfato à baixa temperatura para aplicação como agentes antifúngicos
-----------------------------------------------------------------------------------------
Novas variáveis do treinamento muscular respiratóri